<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
#### flux_get_reflected_1d is same as our new get_reflected_1d in the pushed fluxes.py version
#### get_reflected_1d_original is same as our old get_reflected_1d in  fluxes.py just output changed to whole intensity array
#### not just the top

#### get_reflected_1d_gfluxv is the py version of GFLUXV which doesn't include any post-processing

from new_get_reflected_1d import flux_get_reflected_1d, get_reflected_1d_original
from gfluxv_pyversion import get_reflected_1d_gfluxv

In [10]:
### Benchmarking things between these three routines

## flux_get_reflected_1d gets you level upwelling and downwelling scattering post-processed fluxes 

## get_reflected_1d_original is the original PICASO routine just intensity multiplied with pi to get upwelling fluxes 

## get_reflected_1d_gfluxv is the py version of fortran routine 
## -- it gives you upwelling and downwelling fluxes no post processed 

## now TEST input numbers

numg,numt=1,1 # of angles

nwno=3 # wvpoints

wno=np.array([1.,2.,3.]) # wavelengths

nlevel=11    # of levels
nlayer=nlevel-1 # of layers

tau=np.zeros((nlevel,nwno)) # cumulative tau array
dtau=np.zeros((nlevel-1,nwno)) # per layer tau array
w0,cosb=np.zeros((nlevel-1,nwno)),np.zeros((nlevel-1,nwno)) # ssa and asymmetry
w0+=0.01 # lets assume
cosb+=1e-3 # lets assume
for i in range(nlevel):
    for j in range(nwno):
        tau[i,j]=i+j # just an assumption : tau must always increase downwards

taucld=tau*0+0.1 ## needed for the scattering routines -- not get_reflected_1d_gfluxv
tauray=tau*0+0.1 ## needed for the scattering routines -- not get_reflected_1d_gfluxv



for i in range(nlevel-1):
    dtau[i,:]=tau[i+1,:]-tau[i,:] # incremental opd
 

surf_reflect=1.0 

ubar0=np.zeros((numg,numt))

ubar0+=1.0 # cosine(0)
ubar1=ubar0 #assumed

F0PI = np.zeros(nwno) + 1.0  # assumed


b_top = 0.0   #needed for  get_reflected_1d_gfluxv  as I take it as an input as in GFLUXV                                  
#needed for  get_reflected_1d_gfluxv  as I take it as an input as in GFLUXV
b_surface = 0. + surf_reflect*ubar0[0, 0]*F0PI*np.exp(-tau[-1, :]/ubar0[0, 0]) 


tridiagonal=0

## delta eddington correction for flux_get_reflected_1d, get_reflected_1d_original

tau_og=np.zeros((nlevel,nwno))
dtau_og=np.zeros((nlevel-1,nwno))
w0_og,cosb_og=np.zeros((nlevel-1,nwno)),np.zeros((nlevel-1,nwno))

tau_og=tau
dtau_og=dtau
w0_og,cosb_og=w0,cosb


### now calculate ftau_cld and ftau_ray for flux_get_reflected_1d, get_reflected_1d_original (post-processing routines)
taucld_layer=0.5*(taucld[:-1,:]+taucld[1:,:])
tauray_layer=0.5*(tauray[:-1,:]+tauray[1:,:])

ftau_cld=taucld_layer/(taucld_layer+tauray_layer)
ftau_ray=tauray_layer/(taucld_layer+tauray_layer)

gcos2=0.5*ftau_ray
cos_theta=1.0

## phase function assumptions
frac_a,frac_b,frac_c=1,-1,2

gbar = (w0*taucld_layer/(w0*taucld_layer+tauray_layer))*cosb

gb=-0.5*gbar
f=1-gb**2

constant_back, constant_forward=1-f,f
single_phase=3 ##TTHG_ray
multi_phase=0  ## N=2

In [11]:
## running our new routine for upward and downward flux calculations
flux_minus_all, flux_plus_all, flux_minus_midpt_all,flux_plus_midpt_all=flux_get_reflected_1d(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,gcos2, ftau_cld, ftau_ray,
    dtau_og, tau_og, w0_og, cosb_og, surf_reflect,ubar0, ubar1,cos_theta,F0PI,single_phase, multi_phase,
    frac_a, frac_b, frac_c, constant_back, constant_forward, tridiagonal, calc_type=1)

[0.00157073 0.00057784 0.00021258]


In [12]:
## running our new routine for just upward intensity output at top
intensity_at_top=flux_get_reflected_1d(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,gcos2, ftau_cld, ftau_ray,
    dtau_og, tau_og, w0_og, cosb_og, surf_reflect,ubar0, ubar1,cos_theta,F0PI,single_phase, multi_phase,
    frac_a, frac_b, frac_c, constant_back, constant_forward, tridiagonal, calc_type=0)

[0.00157073 0.00057784 0.00021258]


In [13]:
## running a bit modified old routine for upward flux output at all levels (output array was changed only)
flux_plus_all_original=get_reflected_1d_original(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,gcos2, ftau_cld, ftau_ray,
    dtau_og, tau_og, w0_og, cosb_og, surf_reflect,ubar0, ubar1,cos_theta,F0PI,single_phase, multi_phase,
    frac_a, frac_b, frac_c, constant_back, constant_forward, tridiagonal)

In [14]:
### difference in upward fluxes between the two post-processed routines (original PICASO and new post-processed)



print(flux_plus_all-flux_plus_all_original)

## so our new routine does well with upwelling flux component

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [15]:
## check the calc_type=0 output

print(np.pi*intensity_at_top[0,0,0,:]-flux_plus_all_original[0,0,0,:])

## so the forward component of our module is working just as earlier 

[0. 0. 0.]


In [16]:
flux_minus_all_gfluxv, flux_plus_all_gfluxv, flux_minus_midpt_all_gfluxv, flux_plus_midpt_all_gfluxv=get_reflected_1d_gfluxv(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,
    surf_reflect,b_top,b_surface,ubar0, F0PI,tridiagonal=0,delta_approx=1)

In [17]:
### difference in downwelling level fluxes between post-processed and not post-processed routines

print((flux_minus_all-flux_minus_all_gfluxv)/flux_minus_all_gfluxv)

## about 1% level differences than no post-processing

[[[[ 0.00000000e+00 -9.99999997e-09 -1.99999998e-08]
   [ 1.12359174e-04  1.12349172e-04  1.12339171e-04]
   [ 2.07422628e-03  2.07421625e-03  2.07420623e-03]
   [ 4.92958041e-03  4.92957036e-03  4.92956031e-03]
   [ 8.21832629e-03  8.21831621e-03  8.21830613e-03]
   [ 1.17177022e-02  1.17176921e-02  1.17176820e-02]
   [ 1.53195261e-02  1.53195160e-02  1.53195058e-02]
   [ 1.89709660e-02  1.89709558e-02  1.89709456e-02]
   [ 2.26428133e-02  2.26428031e-02  2.26427928e-02]
   [ 2.62670531e-02  2.62670428e-02  2.62670325e-02]
   [ 2.89862455e-02  2.89862353e-02  2.89862250e-02]]]]


In [8]:
### difference in upwelling level fluxes between post-processed and not post-processed routines

print((flux_plus_all-flux_plus_all_gfluxv)/flux_plus_all_gfluxv)

## about 10% level differences than no post-processing
## also in top atmosphere our module underpredicts flux and in bottom overpredicts flux than gfluxv

[[[[-1.45137272e-01 -1.45137281e-01 -1.45137289e-01]
   [-1.45385661e-01 -1.45385669e-01 -1.45385678e-01]
   [-1.45455553e-01 -1.45455562e-01 -1.45455571e-01]
   [-1.45123370e-01 -1.45123378e-01 -1.45123387e-01]
   [-1.42277383e-01 -1.42277392e-01 -1.42277400e-01]
   [-1.21343655e-01 -1.21343663e-01 -1.21343672e-01]
   [ 2.76158970e-02  2.76158868e-02  2.76158767e-02]
   [ 9.15875862e-01  9.15875845e-01  9.15875828e-01]
   [ 2.25135923e+00  2.25135922e+00  2.25135921e+00]
   [ 1.03681612e+00  1.03681612e+00  1.03681612e+00]
   [ 1.81340806e-08  1.80651939e-08  1.79963075e-08]]]]


In [9]:
### difference in downwelling layer fluxes between post-processed and not post-processed routines

print((flux_minus_midpt_all-flux_minus_midpt_all_gfluxv)/flux_minus_midpt_all_gfluxv)

## about 1% level differences than no post-processing in midpt fluxes

[[[[-1.11131729e-03 -1.11132728e-03 -1.11133727e-03]
   [ 9.29303601e-05  9.29203592e-05  9.29103583e-05]
   [ 2.58169203e-03  2.58168201e-03  2.58167198e-03]
   [ 5.69248141e-03  5.69247135e-03  5.69246129e-03]
   [ 9.10533779e-03  9.10532770e-03  9.10531760e-03]
   [ 1.26650792e-02  1.26650691e-02  1.26650590e-02]
   [ 1.62962475e-02  1.62962373e-02  1.62962271e-02]
   [ 1.99614654e-02  1.99614552e-02  1.99614450e-02]
   [ 2.36322102e-02  2.36321999e-02  2.36321897e-02]
   [ 2.71354671e-02  2.71354569e-02  2.71354466e-02]]]]


In [18]:
### difference in upwelling layer fluxes between post-processed and not post-processed routines

print((flux_plus_midpt_all-flux_plus_midpt_all_gfluxv)/flux_plus_midpt_all_gfluxv)

## about 10% level differences than no post-processing in midpt fluxes
## also in top atmosphere our module underpredicts flux and in bottom overpredicts flux than gfluxv

## however all of these differences are from arbitrary inputs.

[[[[-0.1453023  -0.14530231 -0.14530232]
   [-0.14270114 -0.14270115 -0.14270116]
   [-0.14131066 -0.14131067 -0.14131068]
   [-0.13964597 -0.13964598 -0.13964598]
   [-0.13158399 -0.131584   -0.13158401]
   [-0.07527728 -0.07527729 -0.0752773 ]
   [ 0.30706993  0.30706992  0.30706991]
   [ 1.83949323  1.83949321  1.8394932 ]
   [ 1.7648631   1.7648631   1.76486309]
   [ 0.43340223  0.43340223  0.43340223]]]]
